In [46]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import datetime as dt
import numpy as np

# Load data

In [58]:
files = os.listdir("data")
# Sort dates
dates = [dt.datetime.strptime(re.findall(r'\d{4}-\d{2}-\d{2}',f)[0],'%Y-%m-%d') for f in files]
sorted(dates,reverse=True)
dates = [i.strftime('%Y-%m-%-d') for i in dates]

In [59]:
dfs = {d: pd.read_csv(f'data/worldometers-{d}.tsv',sep="\t") for d in dates}

# Preprocessing

In [61]:
dfs[dates[0]]

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Continent
0,USA,48999737,NaN,798551,NaN,38799986.0,NaN,9401200.0,12259.0,146830.0,2393.0,747324581.0,2239391.0,333717757,North America
1,India,34555431,NaN,467468,NaN,33977830.0,9868.0,110133.0,8944.0,24701.0,334.0,637106009.0,455409.0,1398976181,Asia
2,Brazil,22055608,NaN,613697,NaN,21275209.0,NaN,166702.0,8318.0,102741.0,2859.0,63776166.0,297086.0,214672763,South America
3,UK,10021497,NaN,144433,NaN,8874965.0,NaN,1002099.0,928.0,146546.0,2112.0,357537501.0,5228313.0,68384867,Europe
4,Russia,9502879,34690.0,270292,1235.0,8200971.0,36145.0,1031616.0,2300.0,65078.0,1851.0,222900000.0,1526482.0,146022030,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,Marshall Islands,4,NaN,,NaN,4.0,NaN,0.0,NaN,67.0,NaN,NaN,NaN,59749,Australia/Oceania
220,Samoa,3,NaN,,NaN,3.0,NaN,0.0,NaN,15.0,NaN,NaN,NaN,200258,Australia/Oceania
221,Saint Helena,2,NaN,,NaN,2.0,NaN,0.0,NaN,328.0,NaN,NaN,NaN,6102,Africa
222,Micronesia,1,NaN,,NaN,1.0,NaN,0.0,NaN,9.0,NaN,NaN,NaN,116709,Australia/Oceania


In [52]:
for df in dfs.values():
    df = df.replace(r'^\s*$', np.nan, regex=True)
    for i 